In [1]:
# Imports:
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import matplotlib.cm as cm
import seaborn as sns
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
sns.set_theme()
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
length = 277
import shap

2024-03-22 17:16:25.447614: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 17:16:25.513266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 17:16:25.513322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 17:16:25.515347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-22 17:16:25.528377: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2024-03-22 17:16:26.792803: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 17:16:26.873312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 17:16:26.873366: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 17:16:27.017068: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 17:16:27.017137: I external/local_xla/xla/stream_executor

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8521103841777783846
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8504987100035146831
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-03-22 17:16:27.017187: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 17:16:27.017224: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /device:GPU:0 with 5535 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values

print(train_values.shape)
print(test_values.shape)

X_train = train_values[:,:-2]
X_test = test_values[:,:-2]




y_train = train_values[:,-2]
y_test = test_values[:,-2]

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

(38751, 277)
(72794, 277)


In [4]:
def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed

In [5]:
unique_classes, class_counts = np.unique(test_values[:, -2], return_counts=True)
class_counts_dict = dict(zip(unique_classes, class_counts))
print("Count of each class in test values:")
print(class_counts_dict)


Count of each class in test values:
{1.0: 58951, 2.0: 2493, 3.0: 6725, 4.0: 2923, 5.0: 1277, 6.0: 425}


In [7]:
model_files = ['./models/mlpmodel.h5']
model_names = ['MLP']

for model_file, model_name in zip(model_files, model_names):
        if model_name == 'LSTM':
                model = load_model(model_file, custom_objects={'focal_loss_fixed': focal_loss})
        else:
                model = load_model(model_file)

        feature_importance_df = pd.DataFrame()

        for class_num in range(1, 7):
            N_test = test_values[test_values[:,-2]==class_num]

            if class_num == 1:
                Xs_test = N_test[:10000,:-2]
            else:
                Xs_test = N_test[:,:-2]
            ys_test = N_test[:len(N_test),-2]

            ys_test1 = to_categorical(ys_test)
            explainer = shap.GradientExplainer(model, X_train)
            shap_values = explainer.shap_values(Xs_test)
            shap_values_array = np.array(shap_values[class_num])
            shap_values_array = np.squeeze(shap_values_array)
            shap_values_array.shape
            agg_shap = shap_values_array.mean(0)
            mean_weights = np.zeros((23, ))
            for i in range(0, 275, 12):
                mean_weights[i // 12] = np.mean(agg_shap[i:i+12])

            # Sort based on weight:
            indices_sort = np.argsort(-1 * mean_weights)
            slices = np.arange(1, 24)

            feature_importance_normalized = (mean_weights - mean_weights.min()) / (mean_weights.max() - mean_weights.min())

            # Visualize:
            fig, ax = plt.subplots(1, 2, figsize = (15, 4))
            ax[0].bar(range(23), feature_importance_normalized[indices_sort])
            ax[0].set_title('SHAP weights for class ' + str(class_num) + ' with a '+str(model_name)+' model')
            ax[0].set_xticks(np.arange(23))
            ax[0].set_xticklabels(slices[indices_sort])
            ax[0].set_ylabel('SHAP weight')
            ax[0].set_xlabel('Segment')

            ecg_normalized = (Xs_test[20, :] - Xs_test[20, :].min()) / (Xs_test[20, :].max() - Xs_test[20, :].min())

            ax[1].plot(np.arange(275), ecg_normalized , label = ' ECG for class ' + str(class_num))
            ax[1].plot(np.repeat(feature_importance_normalized, 12), label = 'SHAP weight')
            ax[1].set_title('SHAP weights for class ' + str(class_num) + ' with a '+str(model_name)+' model')
            ax[1].set_ylabel('ECG signal / SHAP weights')
            ax[1].set_xlabel('Time')
            ax[1].legend()
            plt.savefig('./output_pics/SHAP weights for class ' + str(class_num) + ' with a '+str(model_name)+' model', dpi = 400)

            # Append the importance_dict to the feature_importance_df DataFrame
            feature_importance_df['class_' + str(class_num)] = feature_importance_normalized
        # Save the feature_importance_df DataFrame to a CSV file
        feature_importance_df.to_csv('./output_csv/shap_'+ str(model_name) + '.csv', index=False)
                

2024-03-22 17:18:21.278949: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
/home/alinnavas/.local/lib/python3.10/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
Xs_test1 = Xs_test.reshape(-1, X_test.shape[1], 1)


SyntaxError: invalid syntax (3455886725.py, line 1)